# OpenAI Streaming - Traced With LastMile

Four use cases:

1. Text - Non-streaming
2. [this one] Text - Streaming
3. Tool Calls - Non-streaming
4. Tool Calls - Streaming

In [1]:
# !pip install scipy --quiet
# !pip install tenacity --quiet
# !pip install tiktoken --quiet
# !pip install termcolor --quiet
# !pip install openai --quiet
# !pip install "tracing-auto-instrumentation[openai]"

# Create ~/.env file with this line: OPENAI_API_KEY=<your key here>
# You can get your key from https://platform.openai.com/api-keys 
import openai
import dotenv
import os
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
import openai

from lastmile_eval.rag.debugger.api.tracing import LastMileTracer

from tracing_auto_instrumentation.openai import openai as openai_tracing
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="OpenAI Text Calling w. Streaming",
)
openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)
client = openai_tracing.wrap(openai_client, tracer)

/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-29 19:06:59,241 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 19:06:59,323 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=OpenAI+Text+Calling+w.+Streaming HTTP/1.1" 200 367
2024-05-29 19:06:59,325 - load_ssl_context verify=True cert=None trust_env=True http2=False
2024-05-29 19:06:59,326 - load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'


get wrapping...
enter 1
inside openai v1 wrapper
chatv1 wrapper initializing...
chat.completions=<openai.resources.chat.completions.Completions object at 0x1519a0aa0>
chatv1 wrapper regular completions
inside regular embedding wrapper


In [3]:
def run_my_existing_openai_app(user_message: str, stream: bool = True):
    completion_params = {
        "model": "gpt-3.5-turbo",
        "top_p": 1,
        "max_tokens": 3000,
        "temperature": 1,
        "stream": stream,
        "messages": [
            {
                "content": user_message,
                "role": "user",
            }
        ],
    }
    import inspect
    print(inspect.getsource(client.chat.completions.create))

    response = client.chat.completions.create(**completion_params)
    print("Chat Completion Response: ")
    if stream:
        for chunk in response:
            print(f"{chunk=}")
    else:
        print(f"{response=}")

In [4]:
print(hasattr(openai_client, "chat") and hasattr(openai_client.chat, "completions"))

True


In [5]:
# Run your code as usual
stream = False
run_my_existing_openai_app("Tell me a joke about apples", stream=stream)

import inspect 



    def create(self, *args, **kwargs):
        print("V1, about to enter create function")
        return ChatCompletionWrapper(
            self.__completions.create, None, self.tracer
        ).create(*args, **kwargs)

V1, about to enter create function
completion wrapper is initializing...
Chat Completion Response: 
response=<generator object ChatCompletionWrapper.create at 0x151789460>
